Hoja de trabajo 2 DataScience - Series de tiempo

Andree Toledo
Andres Quinto 18288



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.cm as cm
from factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from apyori import apriori
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
pd.set_option('display.max_rows', None)

Analisis Exploratorio

In [ ]:
IMPORT = pd.read_excel('./IMPORTACION-VOLUMEN-2022-05.xlsx', skiprows=6)
CONSUME = pd.read_excel('./CONSUMO-2022-05.xlsx', skiprows=6)
# print(IMPORT.columns)
# print(CONSUME.columns)

Pequeño fix para nombres de la columna

In [ ]:
IMPORT.rename(columns={'Gasolina superior': 'Gasolinasuperior','Gasolina regular': 'Gasolinaregular'}, inplace=True)
CONSUME.rename(columns={'Gasolina superior': 'Gasolinasuperior','Gasolina regular': 'Gasolinaregular'}, inplace=True)

In [ ]:
Relevantes=['Fecha', 'Gasolinasuperior', 'Gasolinaregular', 'Diesel']
SinFecha=['Gasolina superior', 'Gasolina regular', 'Diesel']
consumes = CONSUME[Relevantes]
imports = IMPORT[Relevantes]


# a partir de la linea 264 ya no se toma en relevancia para la interpretacion de datos
CONSUME = CONSUME[:264]
IMPORT = IMPORT[:264]
consumes = consumes[:270]
imports = imports[:270]

In [ ]:
consumes.head(15)

In [ ]:
consumes.info()

In [ ]:
imports.head(15)


In [ ]:
imports.info()

DATOS Y LIMPIEZA

Los data sets contienen un resumen acerca del consumo e importacion de la gasolina en Guatemala en el periodo de enero 2001 hasta junio del 2022.

Informacion como: galones de gasolina de super, regular y diesel consumidos e importados.

EXPLORACION DE DATOS:

Nuestras variables de interes son todas continuas (Super, regular, diesel y fecha). Para determinar su distribucion realizamos sus graficos correspondientes mas histogramas.

Los datos presentan un sesgo hacia la derecha y no una distribucion normal.

PARA IMPORTACION

In [ ]:
for var in Relevantes:
    sns.displot(imports[var], kde=True, color = 'lightcoral')

PARA CONSUMOS

In [ ]:
for var in Relevantes:
    sns.displot(consumes[var], kde=True, color='lightcoral')

In [ ]:
plt.hist(consumes.Gasolinaregular, color="lightcoral")

In [ ]:
plt.hist(consumes.Gasolinasuperior, color="lightcoral")


In [ ]:
plt.hist(consumes.Diesel, color="lightcoral")

In [ ]:
plt.hist(imports.Gasolinaregular, color="lightcoral")

In [ ]:
plt.hist(imports.Gasolinasuperior, color="lightcoral")

In [ ]:
plt.hist(imports.Diesel, color="lightcoral")

In [ ]:
print(IMPORT.head())

In [ ]:
cols=["Fecha"]
IMPORT['date'] = pd.to_datetime(IMPORT[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"), format='%Y-%m', errors='ignore')
CONSUME['date'] = pd.to_datetime(CONSUME[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"))

SERIES CONFORME TIEMPO

Podemos apreciar que cada una de las graficas posee una tendencia, por lo cual no es constante en la media. Los consumos e importacion desde el año 2001 parace ir en aumento conforme los el tiempo. 

Se puede apreciar que los datasets poseen estacionalidad, sin embargo, la varianza en este caso parece no ser constante. 

Para las series de tiempo de importacion podemos apreciar que estas oscilan con mayor frecuencia y de una forma mas variada.

SERIES DE TIEMPO CONSUMOS

In [ ]:
CONSUME.plot(x='date', y='Gasolinasuperior', style='-')

In [ ]:
CONSUME.plot(x='date', y='Gasolinaregular', style='-')

In [ ]:
CONSUME.plot(x='date', y='Diesel', style='-')

SERIES DE IMPORTACION 

In [ ]:
IMPORT.plot(x='date', y='Gasolinasuperior', style='-')

In [ ]:
IMPORT.plot(x='date', y='Gasolinaregular', style='-')

In [ ]:
IMPORT.plot(x='date', y='Diesel', style='-')

CONJUNTO DE ENTRENAMIENTO Y DICK FULLER

Prueba de Dickey-Fuller para las gasolinas

In [ ]:
train = int(len(consumes) * 0.7)
entrenamientoConsumo = consumes[0:train]
PruebaConsumo = consumes[train:len(consumes)]
entrenamientoConsumo = entrenamientoConsumo.set_index(['Fecha'])
PruebaConsumo = PruebaConsumo.set_index(['Fecha'])

In [ ]:
testRegular = entrenamientoConsumo['Gasolinaregular']


testSuper = entrenamientoConsumo['Gasolinasuperior']


testDiesel = entrenamientoConsumo['Diesel']


In [ ]:
dfTest = adfuller(testRegular, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Valor Crítico (%s)'%key] = value
print(salidaDf)

In [ ]:
dfTest = adfuller(testSuper, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Valor Crítico (%s)'%key] = value
print(salidaDf)

In [ ]:
dfTest = adfuller(testDiesel, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Valor Crítico (%s)'%key] = value
print(salidaDf)

Podemos ver que en los p-values obtuvimos todos mayores a 0.05 debido a esto, no se rechaza la hipotesis nula de la existencia de raices unitarias. Podemos concluir que la serie no es estacionaria en media.

Procedemos a descomponer en componentes:
REF:
https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

In [ ]:
DeCompose = seasonal_decompose(testRegular, model='additive' , period=24,  two_sided=True, extrapolate_trend='freq') 
DeCompose.plot()
DeCompose = seasonal_decompose(testSuper,  model='additive' , period=24,  two_sided=True, extrapolate_trend='freq')
DeCompose.plot()
DeCompose = seasonal_decompose(testDiesel, model='additive' , period=24,  two_sided=True, extrapolate_trend='freq')
DeCompose.plot()

Media y desviacion estandar para la gasolina super, regular y diesel

In [ ]:
Origen = plt.plot(testRegular, label="Origen")
media = plt.plot(mediaMovil, label = 'Media')
DE = plt.plot(deMovil, label = 'Desviación Estándar para gasolina regular')
plt.title('Media y desviación estándar')
plt.legend(loc='best')

In [ ]:
Origen = plt.plot(testSuper, label="Origen")
media = plt.plot(mediaMovil, label = 'Media')
DE = plt.plot(deMovil, label = 'Desviación Estándar para gasolina super')
plt.title('Media y desviación estándar')
plt.legend(loc='best')


In [ ]:
Origen = plt.plot(testDiesel, label="Origen")
media = plt.plot(mediaMovil, label = 'Media')
DE = plt.plot(deMovil, label = 'Desviación Estándar para Diesel')
plt.title('Media y desviación estándar')
plt.legend(loc='best')